In [1]:
# Import necessary modules
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, transpile
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
import random
import numpy as np

# BB84 Protocol
This is a simple implementation of the BB84 protocol using Qiskit. The BB84 protocol is a quantum key distribution protocol that allows two parties, Alice and Bob, to share a secret key over an insecure channel. 


In [13]:
# Setting up the environment
n = 28 # Length of the key to be generated

#Alice's operations

circuit = QuantumCircuit(n, n)

alice_key = np.random.randint(0, 2, size = n)   # Random key 
alice_basis = np.random.randint(0, 2, size=n)  # Random basis (0 = Z-basis, 1 = X-basis)


for i, bit in enumerate(alice_key): 
    if bit == 1:        
        circuit.x(i)  


for i, basis in enumerate(alice_basis):
    if basis == 1:
        circuit.h(i) 

print(alice_key)

[1 0 1 0 0 0 1 0 1 1 1 0 1 0 1 0 1 0 0 1 0 0 1 0 0 1 0 0]


In [14]:
# Bob's operations
bob_basis = np.random.randint(0, 2, size=n)

for i, basis in enumerate(bob_basis):
    if basis == 1:
        circuit.h(i)  # Apply Hadamard for diagonal basis

# Bob measures the qubits
circuit.measure(range(n), range(n))

# Run the simulation
simulator = Aer.get_backend('qasm_simulator')
circ = transpile(circuit, simulator)
result = simulator.run(circ, shots = 1).result()
counts = result.get_counts(circ)
#plot_histogram(counts)

In [15]:
total_measurements = sum(counts.values())
print("Total Measurements:", total_measurements)

Total Measurements: 1


In [16]:
most_common_result = max(counts, key=counts.get)
print("Most common result:", most_common_result)

bob_result = most_common_result
bob_result = np.array([int(k) for k in bob_result]) # Convert to an array
bob_result = np.flip(bob_result) # Reverse the order of the bits

Most common result: 1111010011011101011011100101


In [17]:
# Key generation through basis reconciliation
shared_key = []
for i in range(n): #The only communication between A and B is to compare their basis,
    if alice_basis[i] == bob_basis[i]:   #no information about the qubits is exchanged.
        shared_key.append(bob_result[i]) 
shared_key = np.array(shared_key)

# Output results
print("Simulation BB84 Protocol with", n, "qubits")
print("--------------------")
print("Alice's Basis:", alice_basis)
print("Bob's Basis:  ", bob_basis)
print("--------------------")
print("Alice's Key:", alice_key)  
print("Bob's Key:  ", bob_result)
print("Shared Key: ", shared_key)

#print("Alice length of key:", len(alice_key))
#print("Bob length of key:", len(bob_result))

Simulation BB84 Protocol with 28 qubits
--------------------
Alice's Basis: [1 1 1 1 0 0 1 0 0 0 0 1 1 0 1 0 1 0 1 1 0 1 1 1 0 1 1 0]
Bob's Basis:   [0 1 0 0 0 1 1 1 1 1 0 1 0 1 1 1 0 0 0 0 1 1 1 0 1 1 0 1]
--------------------
Alice's Key: [1 0 1 0 0 0 1 0 1 1 1 0 1 0 1 0 1 0 0 1 0 0 1 0 0 1 0 0]
Bob's Key:   [1 0 1 0 0 1 1 1 0 1 1 0 1 0 1 1 1 0 1 1 0 0 1 0 1 1 1 1]
Shared Key:  [0 0 1 1 0 1 0 0 1 1]
Alice length of key: 28
Bob length of key: 28


In [18]:
# Last check to see if the keys are the same
matching_bases = [i for i in range(n) if alice_basis[i] == bob_basis[i]] # Communication only happens regarding the bases, not the qubits

bob_key_shared = [bob_result[i] for i in matching_bases]
alice_key_shared = [alice_key[i] for i in matching_bases]
print("Matching bases:", matching_bases)

print("Alice's key:", alice_key_shared)
print("Bob's key:  ", bob_key_shared)


Matching bases: [1, 4, 6, 10, 11, 14, 17, 21, 22, 25]
Alice's key: [0, 0, 1, 1, 0, 1, 0, 0, 1, 1]
Bob's key:   [0, 0, 1, 1, 0, 1, 0, 0, 1, 1]


In [19]:
# Compare Alice's and Bob's key with a random sample of the key
sample_size = len(matching_bases) // 2
print("Sample size:", sample_size)
random_sample = random.sample(range(len(matching_bases)), sample_size)


bob_sample_key = [bob_key_shared[i] for i in random_sample]
alice_sample_key = [alice_key_shared[i] for i in random_sample]

print("Checking for eavesdropping...")
print("Check done by comparing a random sample of the keys")
print("---------------------------------------------------")

if alice_sample_key == bob_sample_key:
    print("No eavesdropping detected in random sample.")
else:
    print("Eavesdropping detected in random sample. Secure key exchange not possible.")
    

Sample size: 5
Checking for eavesdropping...
Check done by comparing a random sample of the keys
---------------------------------------------------
No eavesdropping detected in random sample.
